In [69]:
import json
import pandas as pd
from pathlib import Path
from takco.util import reform_dict

root = Path('../../output/eval-dbp/')

predictions = []
for split1 in root.iterdir():
    dataset = split1.stem
    for split2 in split1.iterdir():
        linker = split2.stem
        for f in split2.joinpath('2-report').glob('*'):
            for line in open(f):
                report = json.loads(line)
                category = report.get('category')
                for task, scores in report.get('scores', {}).items():
                    predictions.append({
                        'dataset': dataset,
                        'category': category,
                        'linker': linker,
                        'task': task,
                        **scores
                    })


pred = pd.DataFrame.from_dict(predictions).set_index(['dataset', 'category', 'linker','task'])
display.display(pred.style.set_caption('Predictions:'))

In [61]:
import json
import pandas as pd
from pathlib import Path
from takco.util import reform_dict

root = Path('../../output/eval-extract/')

extractions = []
for split1 in root.iterdir():
    linker = '-'.join(split1.stem.split('-')[-2:])
    for f in split1.joinpath('4-report').glob('*'):
        report = json.load(f.open())
        for scores in report.get('novelty', {}):
            extractions.append({'linker':linker, **scores})


extr = pd.DataFrame.from_dict(extractions).set_index(['linker','kb','task','novelty'])
display.display(extr.style.set_caption('Extractions:'))